In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

In [2]:
!pip install -r requirements.txt

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/usr/local/anaconda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/anaconda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout


Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 600, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
8000/8000 [==============================] - 2s 233us/step - loss: 0.4106 - acc: 0.8301
Epoch 2/10
8000/8000 [==============================] - 2s 190us/step - loss: 0.3565 - acc: 0.8540
Epoch 3/10
8000/8000 [==============================] - 2s 188us/step - loss: 0.3484 - acc: 0.8589
Epoch 4/10
8000/8000 [==============================] - 2s 189us/step - loss: 0.3444 - acc: 0.8602
Epoch 5/10
8000/8000 [==============================] - 2s 195us/step - loss: 0.3393 - acc: 0.8612
Epoch 6/10
8000/8000 [==============================] - 2s 202us/step - loss: 0.3395 - acc: 0.8636
Epoch 7/10
8000/8000 [==============================] - 2s 198us/step - loss: 0.3349 - acc: 0.8642
Epoch 8/10
8000/8000 [==============================] - 2s 189us/step - loss: 0.3348 - acc: 0.8619
Epoch 9/10
8000/8000 [==============================] - 2s 195us/step - loss: 0.3303 - acc: 0.8661
Epoch 10/10
8000/8000 [==============================] - 2s 1

In [9]:

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [10]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [11]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [12]:
# Part 4 - Evaluating, Improving and Tuning the ANN

# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 600, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [16]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 600, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [17]:
parameters = {'batch_size': [25, 32, 10],
              'epochs': [10, 30, 15],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [18]:

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/10
7200/7200 [==============================] - 1s 157us/step - loss: 0.4390 - acc: 0.8156
Epoch 2/10
7200/7200 [==============================] - 1s 102us/step - loss: 0.3661 - acc: 0.8508
Epoch 3/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.3517 - acc: 0.8593
Epoch 4/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.3464 - acc: 0.8556
Epoch 5/10
7200/7200 [==============================] - 1s 110us/step - loss: 0.3426 - acc: 0.8639
Epoch 6/10
7200/7200 [==============================] - 1s 107us/step - loss: 0.3408 - acc: 0.8621
Epoch 7/10
7200/7200 [==============================] - 1s 105us/step - loss: 0.3405 - acc: 0.8626
Epoch 8/10
7200/7200 [==============================] - 1s 104us/step - loss: 0.3382 - acc: 0.8601
Epoch 9/10
7200/7200 [==============================] - 1s 110us/step - loss: 0.3351 - acc: 0

Epoch 3/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.3564 - acc: 0.8543
Epoch 4/10
7200/7200 [==============================] - 1s 112us/step - loss: 0.3484 - acc: 0.8574
Epoch 5/10
7200/7200 [==============================] - 1s 108us/step - loss: 0.3450 - acc: 0.8576
Epoch 6/10
7200/7200 [==============================] - 1s 111us/step - loss: 0.3445 - acc: 0.8608
Epoch 7/10
7200/7200 [==============================] - 1s 109us/step - loss: 0.3409 - acc: 0.8612
Epoch 8/10
7200/7200 [==============================] - 1s 108us/step - loss: 0.3387 - acc: 0.8607
Epoch 9/10
7200/7200 [==============================] - 1s 113us/step - loss: 0.3385 - acc: 0.8628
Epoch 10/10
7200/7200 [==============================] - 1s 107us/step - loss: 0.3383 - acc: 0.8642
Epoch 1/10
7200/7200 [==============================] - 2s 257us/step - loss: 0.4338 - acc: 0.8189
Epoch 2/10
7200/7200 [==============================] - 1s 107us/step - loss: 0.3813 - acc: 0.8457
Epoch 3/1

7200/7200 [==============================] - 1s 124us/step - loss: 0.3021 - acc: 0.8783
Epoch 28/30
7200/7200 [==============================] - 1s 122us/step - loss: 0.3017 - acc: 0.8781
Epoch 29/30
7200/7200 [==============================] - 1s 119us/step - loss: 0.3019 - acc: 0.8778
Epoch 30/30
7200/7200 [==============================] - 1s 125us/step - loss: 0.2977 - acc: 0.8799
Epoch 1/30
7200/7200 [==============================] - 3s 396us/step - loss: 0.4508 - acc: 0.8139
Epoch 2/30
7200/7200 [==============================] - 1s 121us/step - loss: 0.3665 - acc: 0.8512
Epoch 3/30
7200/7200 [==============================] - 1s 118us/step - loss: 0.3514 - acc: 0.8567
Epoch 4/30
7200/7200 [==============================] - 1s 118us/step - loss: 0.3442 - acc: 0.8612
Epoch 5/30
7200/7200 [==============================] - 1s 119us/step - loss: 0.3429 - acc: 0.8596
Epoch 6/30
7200/7200 [==============================] - 1s 122us/step - loss: 0.3412 - acc: 0.8639
Epoch 7/30
7200/72

7200/7200 [==============================] - 1s 135us/step - loss: 0.3307 - acc: 0.8639
Epoch 12/30
7200/7200 [==============================] - 1s 135us/step - loss: 0.3310 - acc: 0.8662
Epoch 13/30
7200/7200 [==============================] - 1s 137us/step - loss: 0.3279 - acc: 0.8650
Epoch 14/30
7200/7200 [==============================] - 1s 134us/step - loss: 0.3240 - acc: 0.8671
Epoch 15/30
7200/7200 [==============================] - 1s 137us/step - loss: 0.3214 - acc: 0.8687
Epoch 16/30
7200/7200 [==============================] - 1s 138us/step - loss: 0.3218 - acc: 0.8715
Epoch 17/30
7200/7200 [==============================] - 1s 136us/step - loss: 0.3205 - acc: 0.8699
Epoch 18/30
7200/7200 [==============================] - 1s 136us/step - loss: 0.3194 - acc: 0.8706
Epoch 19/30
7200/7200 [==============================] - 1s 139us/step - loss: 0.3150 - acc: 0.8694
Epoch 20/30
7200/7200 [==============================] - 1s 134us/step - loss: 0.3153 - acc: 0.8714
Epoch 21/30


7200/7200 [==============================] - 1s 126us/step - loss: 0.3158 - acc: 0.8722
Epoch 26/30
7200/7200 [==============================] - 1s 128us/step - loss: 0.3156 - acc: 0.8735
Epoch 27/30
7200/7200 [==============================] - 1s 124us/step - loss: 0.3150 - acc: 0.8744
Epoch 28/30
7200/7200 [==============================] - 1s 122us/step - loss: 0.3135 - acc: 0.8769
Epoch 29/30
7200/7200 [==============================] - 1s 125us/step - loss: 0.3137 - acc: 0.8721
Epoch 30/30
7200/7200 [==============================] - 1s 126us/step - loss: 0.3133 - acc: 0.8761
Epoch 1/30
7200/7200 [==============================] - 3s 479us/step - loss: 0.4378 - acc: 0.8126
Epoch 2/30
7200/7200 [==============================] - 1s 126us/step - loss: 0.3868 - acc: 0.8444
Epoch 3/30
7200/7200 [==============================] - 1s 125us/step - loss: 0.3595 - acc: 0.8551
Epoch 4/30
7200/7200 [==============================] - 1s 124us/step - loss: 0.3526 - acc: 0.8571
Epoch 5/30
7200/

...

7200/7200 [==============================] - 6s 864us/step - loss: 0.3495 - acc: 0.8581
Epoch 5/30
7200/7200 [==============================] - 6s 857us/step - loss: 0.3478 - acc: 0.8578
Epoch 6/30
7200/7200 [==============================] - 6s 861us/step - loss: 0.3439 - acc: 0.8596
Epoch 7/30
7200/7200 [==============================] - 6s 863us/step - loss: 0.3408 - acc: 0.8603
Epoch 8/30
7200/7200 [==============================] - 6s 859us/step - loss: 0.3385 - acc: 0.8601
Epoch 9/30
7200/7200 [==============================] - 6s 856us/step - loss: 0.3382 - acc: 0.8604
Epoch 10/30
7200/7200 [==============================] - 6s 858us/step - loss: 0.3350 - acc: 0.8639
Epoch 11/30
7200/7200 [==============================] - 6s 862us/step - loss: 0.3308 - acc: 0.8639
Epoch 12/30
7200/7200 [==============================] - 6s 854us/step - loss: 0.3296 - acc: 0.8661
Epoch 13/30
7200/7200 [==============================] - 6s 854us/step - loss: 0.3290 - acc: 0.8665
Epoch 14/30
7200/

7200/7200 [==============================] - 7s 906us/step - loss: 0.3127 - acc: 0.8696
Epoch 19/30
7200/7200 [==============================] - 7s 909us/step - loss: 0.3072 - acc: 0.8765
Epoch 20/30
7200/7200 [==============================] - 6s 900us/step - loss: 0.3058 - acc: 0.8747
Epoch 21/30
7200/7200 [==============================] - 7s 904us/step - loss: 0.3035 - acc: 0.8761
Epoch 22/30
7200/7200 [==============================] - 7s 904us/step - loss: 0.2992 - acc: 0.8786
Epoch 23/30
7200/7200 [==============================] - 6s 899us/step - loss: 0.2982 - acc: 0.8789
Epoch 24/30
7200/7200 [==============================] - 6s 902us/step - loss: 0.2983 - acc: 0.8771
Epoch 25/30
7200/7200 [==============================] - 7s 905us/step - loss: 0.2967 - acc: 0.8800
Epoch 26/30
7200/7200 [==============================] - 7s 906us/step - loss: 0.2934 - acc: 0.8807
Epoch 27/30
7200/7200 [==============================] - 6s 898us/step - loss: 0.2917 - acc: 0.8806
Epoch 28/30


7200/7200 [==============================] - 7s 920us/step - loss: 0.3621 - acc: 0.8529
Epoch 3/15
7200/7200 [==============================] - 7s 921us/step - loss: 0.3507 - acc: 0.8560
Epoch 4/15
7200/7200 [==============================] - 7s 919us/step - loss: 0.3473 - acc: 0.8578
Epoch 5/15
7200/7200 [==============================] - 7s 919us/step - loss: 0.3453 - acc: 0.8610
Epoch 6/15
7200/7200 [==============================] - 7s 920us/step - loss: 0.3408 - acc: 0.8590
Epoch 7/15
7200/7200 [==============================] - 7s 920us/step - loss: 0.3370 - acc: 0.8647
Epoch 8/15
7200/7200 [==============================] - 7s 918us/step - loss: 0.3367 - acc: 0.8628
Epoch 9/15
7200/7200 [==============================] - 7s 925us/step - loss: 0.3336 - acc: 0.8637
Epoch 10/15
7200/7200 [==============================] - 7s 919us/step - loss: 0.3318 - acc: 0.8640
Epoch 11/15
7200/7200 [==============================] - 7s 915us/step - loss: 0.3299 - acc: 0.8653
Epoch 12/15
7200/72

7200/7200 [==============================] - 35s 5ms/step - loss: 0.4178 - acc: 0.8269
Epoch 2/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3652 - acc: 0.8525
Epoch 3/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3515 - acc: 0.8528
Epoch 4/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3500 - acc: 0.8569
Epoch 5/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3475 - acc: 0.8571
Epoch 6/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3482 - acc: 0.8601
Epoch 7/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3441 - acc: 0.8610
Epoch 8/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3404 - acc: 0.8621
Epoch 9/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3450 - acc: 0.8607
Epoch 10/15
7200/7200 [==============================] - 8s 1ms/step - loss: 0.3403 - acc: 0.8650
Epoch 11/15
7200/7200 [================

7200/7200 [==============================] - 36s 5ms/step - loss: 0.4240 - acc: 0.8190
Epoch 2/15
7200/7200 [==============================] - 7s 988us/step - loss: 0.3551 - acc: 0.8551
Epoch 3/15
7200/7200 [==============================] - 7s 995us/step - loss: 0.3473 - acc: 0.8560
Epoch 4/15
7200/7200 [==============================] - 7s 983us/step - loss: 0.3427 - acc: 0.8603
Epoch 5/15
7200/7200 [==============================] - 7s 989us/step - loss: 0.3412 - acc: 0.8600
Epoch 6/15
7200/7200 [==============================] - 7s 993us/step - loss: 0.3379 - acc: 0.8618
Epoch 7/15
7200/7200 [==============================] - 7s 988us/step - loss: 0.3345 - acc: 0.8617
Epoch 8/15
7200/7200 [==============================] - 7s 980us/step - loss: 0.3351 - acc: 0.8615
Epoch 9/15
7200/7200 [==============================] - 7s 980us/step - loss: 0.3308 - acc: 0.8661
Epoch 10/15
7200/7200 [==============================] - 7s 995us/step - loss: 0.3286 - acc: 0.8662
Epoch 11/15
7200/7200

In [1]:
best_accuracy

NameError: name 'best_accuracy' is not defined

In [20]:
best_parameters

{'batch_size': 32, 'epochs': 30, 'optimizer': 'rmsprop'}

In [13]:
mean

0.8583749942481518

In [14]:
variance

0.008386185646437179

In [ ]:
# Depois de algumas horas:
# best_accuracy 0.8605 
# best_parameters {'batch_size': 32, 'epochs': 30, 'optimizer': 'rmsprop'}
# mean = 0.8583749942481518
# variance = 0.008386185646437179